In [ ]:
import os
import webbrowser

import numpy as np
import pandas as pd
from pyvis.network import Network

## Загрузка данных

In [ ]:
df = pd.read_excel("test.xlsx")
df = df.map(lambda x: np.nan if isinstance(x, str) else x).astype(float)

## Параметры для построения

In [ ]:
# ============================================================================
# НАСТРАИВАЕМЫЕ ПАРАМЕТРЫ
# ============================================================================

# 1. Порог для отображения связей (по абсолютному значению)
THRESHOLD = 0.1  # Минимальная корреляция для отображения связи

# 2. Границы для классификации корреляций
STRONG_POSITIVE_THRESHOLD = 0.6  # Выше этого значения - сильная положительная
WEAK_POSITIVE_THRESHOLD = 0.3  # Выше этого значения - слабая положительная
WEAK_NEGATIVE_THRESHOLD = -0.3  # Ниже этого значения - слабая отрицательная
STRONG_NEGATIVE_THRESHOLD = -0.6  # Ниже этого значения - сильная отрицательная

# 3. Цвета для разных типов корреляций (в формате rgba)
STRONG_POSITIVE_COLOR = "rgba(220, 53, 69, {alpha})"  # Темно-красный
WEAK_POSITIVE_COLOR = "rgba(255, 99, 132, {alpha})"  # Розово-красный
NEAR_ZERO_COLOR = "rgba(150, 150, 150, {alpha})"  # Серый
WEAK_NEGATIVE_COLOR = "rgba(100, 149, 237, {alpha})"  # Голубой
STRONG_NEGATIVE_COLOR = "rgba(25, 25, 112, {alpha})"  # Темно-синий

# 4. Параметры прозрачности (альфа-канал)
MIN_ALPHA = 0.3  # Минимальная прозрачность (самые слабые связи)
MAX_ALPHA = 0.7  # Максимальная прозрачность (самые сильные связи)

# 5. Параметры ширины линий
MIN_WIDTH = 1  # Минимальная ширина линии
MAX_WIDTH = 9  # Максимальная ширина линии

# 6. Параметры размера узлов
MIN_NODE_SIZE = 20  # Минимальный размер узла
MAX_NODE_SIZE = 50  # Максимальный размер узла
SIZE_DIFFERENCE_FACTOR = (
    1.0  # Фактор различия размеров (1.0 = нормально, 2.0 = больше различия)
)

# 7. Цвет узлов по умолчанию (если не заданы кастомные цвета)
DEFAULT_NODE_COLOR = "#4A90E2"  # Синий

# 8. Настройки графа
GRAPH_HEIGHT = "800px"
GRAPH_WIDTH = "100%"
BACKGROUND_COLOR = "#ffffff"
FONT_COLOR = "#333333"

# 9. Имя выходного файла
OUTPUT_FILENAME = "correlation_network_colored.html"

# 10. Настройки физики (если граф "пляшет", уменьшите gravity)
PHYSICS_GRAVITY = -50

# 11. Кастомные цвета для узлов (опционально)
# Ключ - название колонки, значение - цвет в формате HEX или название
CUSTOM_NODE_COLORS = {
    # Пример:
    # 'Feature_0': '#FF6B6B',  # Красный
    # 'Feature_1': '#4ECDC4',  # Бирюзовый
    # 'Feature_2': '#FFD166',  # Желтый
    # 'Feature_3': '#06D6A0',  # Зеленый
    # 'Feature_4': '#118AB2',  # Синий
    # Оставьте пустым, если хотите использовать DEFAULT_NODE_COLOR для всех
}

# 12. Форма узлов по умолчанию
DEFAULT_NODE_SHAPE = "dot"  # Кружок по умолчанию

# 13. Кастомные формы узлов (опционально)
# Ключ - название колонки, значение - форма узла
# Доступные формы: dot, circle, ellipse, diamond, box, star, triangle, triangleDown, square
CUSTOM_NODE_SHAPES = {
    # Пример:
    # 'Feature_0': 'star',
    # 'Feature_1': 'triangle',
    # 'Feature_2': 'square',
    # Оставьте пустым, если хотите использовать DEFAULT_NODE_SHAPE для всех
}

## Построение графа

In [ ]:
# ============================================================================
# ОСНОВНОЙ КОД (не трогать ниже этой линии без необходимости)
# ============================================================================

# Предположим, df уже определен

# Корреляционная матрица (БЕЗ abs() - сохраняем знаки!)
corr_matrix = df.corr()

# Создаем сеть с включенным манипулированием для лучшей производительности
net = Network(
    height=GRAPH_HEIGHT,
    width=GRAPH_WIDTH,
    bgcolor=BACKGROUND_COLOR,
    font_color=FONT_COLOR,
    directed=False,
    notebook=False,  # Если не в Jupyter
)

# Считаем связи для каждого узла (по абсолютному значению)
connections_count = dict.fromkeys(df.columns, 0)

# Проходим по всем парам колонок
edges_to_add = []
for i in range(len(df.columns)):
    for j in range(i + 1, len(df.columns)):
        corr = corr_matrix.iloc[i, j]
        abs_corr = abs(corr)

        # Используем абсолютное значение для порога
        if abs_corr > THRESHOLD:
            col1, col2 = df.columns[i], df.columns[j]
            connections_count[col1] += 1
            connections_count[col2] += 1
            edges_to_add.append((col1, col2, corr))

# Нормализуем размеры узлов с учетом SIZE_DIFFERENCE_FACTOR
max_connections = max(connections_count.values()) if connections_count else 1
min_connections = min(connections_count.values()) if connections_count else 0


# Функция для определения цвета и прозрачности связи
def get_edge_properties(corr_value):
    """
    Возвращает цвет и ширину связи в зависимости от корреляции
    с использованием заданных пользователем параметров.
    """
    abs_corr = abs(corr_value)

    # Прозрачность: чем сильнее связь, тем меньше прозрачность
    alpha = MIN_ALPHA + abs_corr * (MAX_ALPHA - MIN_ALPHA)

    # Определяем цвет в зависимости от знака и силы корреляции
    if corr_value > STRONG_POSITIVE_THRESHOLD:  # Сильная положительная
        color = STRONG_POSITIVE_COLOR.format(alpha=alpha)
    elif corr_value > WEAK_POSITIVE_THRESHOLD:  # Слабая положительная
        color = WEAK_POSITIVE_COLOR.format(alpha=alpha)
    elif corr_value > WEAK_NEGATIVE_THRESHOLD:  # Около нуля
        color = NEAR_ZERO_COLOR.format(alpha=alpha)
    elif corr_value > STRONG_NEGATIVE_THRESHOLD:  # Слабая отрицательная
        color = WEAK_NEGATIVE_COLOR.format(alpha=alpha)
    else:  # Сильная отрицательная
        color = STRONG_NEGATIVE_COLOR.format(alpha=alpha)

    # Ширина пропорциональна силе корреляции
    width = MIN_WIDTH + abs_corr * (MAX_WIDTH - MIN_WIDTH)

    return color, width


# Добавляем узлы
for col in df.columns:
    # Расчет размера узла с учетом SIZE_DIFFERENCE_FACTOR
    size_range = MAX_NODE_SIZE - MIN_NODE_SIZE

    if max_connections > min_connections:
        # Усиливаем различия между узлами
        normalized = (
            (connections_count[col] - min_connections)
            / (max_connections - min_connections)
        ) ** SIZE_DIFFERENCE_FACTOR
    else:
        normalized = 0

    size = MIN_NODE_SIZE + normalized * size_range

    # Используем кастомный цвет если задан, иначе дефолтный
    node_color = CUSTOM_NODE_COLORS.get(col, DEFAULT_NODE_COLOR)

    # Используем кастомную форму если задана, иначе дефолтную
    node_shape = CUSTOM_NODE_SHAPES.get(col, DEFAULT_NODE_SHAPE)

    # Формируем заголовок с информацией о цвете и форме
    color_info = ""
    if CUSTOM_NODE_COLORS and col in CUSTOM_NODE_COLORS:
        color_info = f"<br>Цвет: {CUSTOM_NODE_COLORS[col]}"

    shape_info = ""
    if CUSTOM_NODE_SHAPES and col in CUSTOM_NODE_SHAPES:
        shape_info = f"<br>Форма: {CUSTOM_NODE_SHAPES[col]}"

    title = f"{col}<br>Связей: {connections_count[col]}<br>Размер: {size:.1f}{color_info}{shape_info}"

    # Добавляем узел
    net.add_node(
        col,
        size=size,
        title=title,
        color=node_color,
        shape=node_shape,  # Добавляем параметр формы
    )


# Добавляем ребра с цветами в зависимости от знака корреляции
for col1, col2, corr in edges_to_add:
    edge_color, edge_width = get_edge_properties(corr)

    # Определяем знак для отображения
    sign = "+" if corr > 0 else ""

    net.add_edge(
        col1,
        col2,
        width=edge_width,
        title=f"Корреляция: {sign}{corr:.3f}",
        color=edge_color,
    )

# Настраиваем физику
net.force_atlas_2based(gravity=PHYSICS_GRAVITY)

# Генерируем HTML
html_content = net.generate_html()

# Создаем динамическую легенду на основе заданных параметров
legend_content = f"""
<div style="position: absolute; top: 20px; right: 20px; background: white; 
            padding: 15px; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            font-family: Arial, sans-serif; font-size: 12px; z-index: 1000; max-width: 300px;">
    <h4 style="margin-top: 0; margin-bottom: 10px;">Легенда связей:</h4>
    <div style="margin-bottom: 6px;">
        <span style="display: inline-block; width: 20px; height: 3px; 
                     background-color: rgba(220, 53, 69, 0.9); margin-right: 8px;"></span>
        <span>Сильная положительная (>{STRONG_POSITIVE_THRESHOLD})</span>
    </div>
    <div style="margin-bottom: 6px;">
        <span style="display: inline-block; width: 20px; height: 3px; 
                     background-color: rgba(255, 99, 132, 0.7); margin-right: 8px;"></span>
        <span>Слабая положительная ({WEAK_POSITIVE_THRESHOLD}-{STRONG_POSITIVE_THRESHOLD})</span>
    </div>
    <div style="margin-bottom: 6px;">
        <span style="display: inline-block; width: 20px; height: 3px; 
                     background-color: rgba(150, 150, 150, 0.5); margin-right: 8px;"></span>
        <span>Около нуля ({WEAK_NEGATIVE_THRESHOLD}-{WEAK_POSITIVE_THRESHOLD})</span>
    </div>
    <div style="margin-bottom: 6px;">
        <span style="display: inline-block; width: 20px; height: 3px; 
                     background-color: rgba(100, 149, 237, 0.7); margin-right: 8px;"></span>
        <span>Слабая отрицательная ({STRONG_NEGATIVE_THRESHOLD}-{WEAK_NEGATIVE_THRESHOLD})</span>
    </div>
    <div style="margin-bottom: 15px;">
        <span style="display: inline-block; width: 20px; height: 3px; 
                     background-color: rgba(25, 25, 112, 0.9); margin-right: 8px;"></span>
        <span>Сильная отрицательная (<{STRONG_NEGATIVE_THRESHOLD})</span>
    </div>
    
    <div style="margin-bottom: 10px;">
        <strong>Настройки узлов:</strong><br>
        Размер узлов: {MIN_NODE_SIZE}-{MAX_NODE_SIZE}<br>
        Фактор различия: {SIZE_DIFFERENCE_FACTOR}<br>
        Цвет по умолчанию: {DEFAULT_NODE_COLOR}
    </div>

    <div style="margin-bottom: 10px;">
        <strong>Настройки связей:</strong><br>
        Порог отображения: {THRESHOLD}<br>
        Всего связей: {len(edges_to_add)}<br>
        Минимальная прозрачность: {MIN_ALPHA}<br>
        Ширина линий: {MIN_WIDTH}-{MAX_WIDTH}
    </div>

    <div style="margin-bottom: 10px;">
        <strong>Кастомные цвета узлов:</strong><br>
        {"Да (" + str(len(CUSTOM_NODE_COLORS)) + " узлов)" if CUSTOM_NODE_COLORS else "Нет"}
    </div>

    <hr>
    <div>
        <strong>Управление:</strong><br>
        • Перетаскивание: зажмите фон<br>
        • Масштаб: колесико мыши<br>
        • Информация: наведите на узел или связь
    </div>
</div>
"""

# Вставляем легенду в HTML
html_content = html_content.replace("</body>", legend_content + "</body>")

# Сохраняем в файл
with open(OUTPUT_FILENAME, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"Граф сохранен в {OUTPUT_FILENAME}")

# Выводим статистику с учетом заданных параметров
print("\n" + "=" * 60)
print("СТАТИСТИКА КОРРЕЛЯЦИЙ")
print("=" * 60)
print(f"Всего узлов: {len(df.columns)}")
print(f"Порог корреляции (по модулю): {THRESHOLD}")
print(f"Всего связей (ребер): {len(edges_to_add)}")
print(f"Фактор различия размеров: {SIZE_DIFFERENCE_FACTOR}")
print(f"Цвет узлов по умолчанию: {DEFAULT_NODE_COLOR}")

if CUSTOM_NODE_COLORS:
    print(f"\nКастомные цвета узлов ({len(CUSTOM_NODE_COLORS)} из {len(df.columns)}):")
    for col, color in CUSTOM_NODE_COLORS.items():
        print(f"  {col}: {color}")

# Статистика по заданным категориям
strong_pos = len([c for _, _, c in edges_to_add if c > STRONG_POSITIVE_THRESHOLD])
weak_pos = len(
    [
        c
        for _, _, c in edges_to_add
        if WEAK_POSITIVE_THRESHOLD < c <= STRONG_POSITIVE_THRESHOLD
    ]
)
near_zero = len(
    [
        c
        for _, _, c in edges_to_add
        if WEAK_NEGATIVE_THRESHOLD <= c <= WEAK_POSITIVE_THRESHOLD
    ]
)
weak_neg = len(
    [
        c
        for _, _, c in edges_to_add
        if STRONG_NEGATIVE_THRESHOLD < c < WEAK_NEGATIVE_THRESHOLD
    ]
)
strong_neg = len([c for _, _, c in edges_to_add if c <= STRONG_NEGATIVE_THRESHOLD])

print("\nРаспределение по категориям:")
print(f"  • Сильные положительные (> {STRONG_POSITIVE_THRESHOLD}): {strong_pos}")
print(
    f"  • Слабые положительные ({WEAK_POSITIVE_THRESHOLD} - {STRONG_POSITIVE_THRESHOLD}): {weak_pos}"
)
print(
    f"  • Около нуля ({WEAK_NEGATIVE_THRESHOLD} - {WEAK_POSITIVE_THRESHOLD}): {near_zero}"
)
print(
    f"  • Слабые отрицательные ({STRONG_NEGATIVE_THRESHOLD} - {WEAK_NEGATIVE_THRESHOLD}): {weak_neg}"
)
print(f"  • Сильные отрицательные (< {STRONG_NEGATIVE_THRESHOLD}): {strong_neg}")

# Подсчет общих положительных/отрицательных
positive = len([c for _, _, c in edges_to_add if c > 0])
negative = len([c for _, _, c in edges_to_add if c < 0])
print("\nОбщие:")
print(f"  • Положительных корреляций: {positive}")
print(f"  • Отрицательных корреляций: {negative}")

print("\nКоличество связей по узлам:")
for col in df.columns:
    color_info = (
        f" ({CUSTOM_NODE_COLORS.get(col, 'default')})" if CUSTOM_NODE_COLORS else ""
    )
    print(f"  {col}: {connections_count[col]} связей{color_info}")

print("\n" + "=" * 60)
print("НАСТРОЙКИ АКТИВНЫ:")
print("=" * 60)
print(f"Порог отображения: {THRESHOLD}")
print("Границы категорий:")
print(f"  • Сильная положительная: > {STRONG_POSITIVE_THRESHOLD}")
print(
    f"  • Слабая положительная: {WEAK_POSITIVE_THRESHOLD} - {STRONG_POSITIVE_THRESHOLD}"
)
print(f"  • Около нуля: {WEAK_NEGATIVE_THRESHOLD} - {WEAK_POSITIVE_THRESHOLD}")
print(
    f"  • Слабые отрицательные: {STRONG_NEGATIVE_THRESHOLD} - {WEAK_NEGATIVE_THRESHOLD}"
)
print(f"  • Сильные отрицательные: < {STRONG_NEGATIVE_THRESHOLD}")
print(f"Фактор различия размеров: {SIZE_DIFFERENCE_FACTOR}")
print(f"Цвет узлов по умолчанию: {DEFAULT_NODE_COLOR}")
print(f"Кастомные цвета узлов: {'ДА' if CUSTOM_NODE_COLORS else 'НЕТ'}")

## Открытие в браузере

In [ ]:
# Автоматическое открытие в браузере (опционально)
try:
    webbrowser.open(f"file://{os.path.abspath(OUTPUT_FILENAME)}")
    print("\nФайл открывается в браузере...")
except Exception as e:
    print(f"\nОткройте файл вручную ({e!s}): {os.path.abspath(OUTPUT_FILENAME)}")